# Prerequisites
Need to have Gurobi Optimizer and obtain license.

1. Install Gurobi
- https://www.gurobi.com/downloads/gurobi-software/
- https://support.gurobi.com/hc/en-us/articles/4534161999889-How-do-I-install-Gurobi-Optimizer
2. Obtain License
- https://portal.gurobi.com/iam/licenses/list/


%pip install gurobipy

In [ ]:
# Imports
import os
import json
import numpy as np
import networkx as nx
from natsort import natsorted
import gurobipy as gp
from math import sqrt

In [ ]:
def read_nxgraph(filename: str) -> nx.Graph(): # type: ignore
    graph = nx.Graph()
    with open(filename, 'r') as file:
        # lines = []
        line = file.readline()
        is_first_line = True
        while line is not None and line != '':
            if '//' not in line:
                if is_first_line:
                    strings = line.split(" ")
                    num_nodes = int(strings[0])
                    num_edges = int(strings[1])
                    nodes = list(range(num_nodes))
                    graph.add_nodes_from(nodes)
                    is_first_line = False
                else:
                    node1, node2, weight = line.split()
                    graph.add_edge(int(node1), int(node2), weight=weight)
            line = file.readline()
    return graph

In [ ]:
def gurobi_maxcut(graph):

    # Create QUBO matrix
    nodes = len(list(graph.nodes))
    J = nx.to_numpy_array(graph)

    # Construct gurobi model
    model = gp.Model("maxcut_qubo")

    # Set time limit 1 hr
    model.Params.LogFile = "../logs/gurobi.log"
    model.setParam('TimeLimit', 3600)
    model.Params.LogToConsole = 1
    model.setParam("MIPGap", 0.0)

    # Create variable for each vertex
    x = model.addVars(nodes, vtype=gp.GRB.BINARY)

    objective = gp.quicksum(
        -J[i, j] * (2 * x[i] - 1) * (2 * x[j] - 1) * (1/sqrt(nodes))
        for i in range(nodes) for j in range(i+1, nodes) if J[i, j] != 0.0
    )
    model.setObjective(objective, gp.GRB.MINIMIZE)
    
    # Solve
    model.optimize()
    obj_val = model.ObjVal
    obj_bnd = model.ObjBound
    solution = ""
    for i in range(nodes):
        solution += f"{int(x[i].X)}"

    return obj_val, solution

In [ ]:
# graph = read_nxgraph("../data/vna/ER/100_SK_seed40.txt")
# obj_val, sol = gurobi_maxcut(graph)

In [ ]:
# Parse all files in data_dir 
data_dir = f"../data/vna/SK"
input_files = [ f for f in os.listdir(data_dir) ]
input_files = natsorted(input_files)

# Write directories & files
results_dir = f"../results/{'/'.join(data_dir.split('/')[2:])}"
solutions_dir = f"../solutions/{'/'.join(data_dir.split('/')[2:])}"
os.makedirs(results_dir, exist_ok=True)
os.makedirs(solutions_dir, exist_ok=True)

i = 0
for file in input_files:
    i += 1
    if i > 25: break

    graph = read_nxgraph(f'{data_dir}/{file}') 
    obj_val, sol = gurobi_maxcut(graph)

    with open(f"{results_dir}/GUROBI.txt", "a") as f:
        f.write(f"{obj_val}\n")

    with open(f"{solutions_dir}/GUROBI.txt", "a") as f:
        f.write(f"{sol}\n")


 4081189 3127413   -8.84763   56   71   -7.15263   -8.84763  23.7%  29.7 1190s
